# "Churn de Clientes" na Telecom X

O "Churn de Clientes" da Telecom X, tem como objetivo principal entender os fatores que contribuem para o alto índice de cancelamentos de clientes (churn). O objetivo aqui é coletar, tratar e analisar os dados usando o Python e suas bibliotecas.

Os insights gerados a partir desta análise serão cruciais para que a equipe de Data Science possa desenvolver modelos preditivos e estratégias eficazes para reduzir a evasão de clientes.
___

# 🔍 **Análise da Estrutura dos Dados**
Os dados são organizados em um formato hierárquico (dicionário dentro de dicionário), contendo as seguintes informações:

1. **Identificação do Cliente**  
   - `'customerID'`: Identificador único do cliente.  
   - `'Churn'`: Indica se o cliente cancelou o serviço (`Yes` ou `No`).  

2. **Informações do Cliente** (`'customer'`)  
   - `'gender'`: Gênero do cliente (`Male` ou `Female`).  
   - `'SeniorCitizen'`: Indica se o cliente é idoso (0 = Não, 1 = Sim).  
   - `'Partner'`: Se o cliente tem um parceiro(a).  
   - `'Dependents'`: Se o cliente tem dependentes.  
   - `'tenure'`: Tempo de permanência como cliente (em meses).  

3. **Serviços de Telefonia** (`'phone'`)  
   - `'PhoneService'`: Indica se o cliente possui serviço de telefone (`Yes` ou `No`).  
   - `'MultipleLines'`: Se possui múltiplas linhas telefônicas.  

4. **Serviços de Internet** (`'internet'`)  
   - Tipo de serviço de internet contratado (`DSL`, `Fiber optic`, `No`).  
   - Serviços adicionais (`OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`).  

5. **Informações da Conta** (`'account'`)  
   - `'Contract'`: Tipo de contrato (`Month-to-month`, `One year`, `Two year`).  
   - `'PaperlessBilling'`: Se o cliente recebe faturas eletrônicas.  
   - `'PaymentMethod'`: Método de pagamento (`Bank transfer`, `Credit card`, `Electronic check`, `Mailed check`).  
   - `'Charges'`:  
     - `'Monthly'`: Valor mensal cobrado.  
     - `'Total'`: Valor total pago pelo cliente.  



#📌 Extracão

In [ ]:
import requests
import pandas as pd

In [ ]:
url = 'https://raw.githubusercontent.com/WBruni/analise-churn-telecom-x/refs/heads/main/TelecomX_Data.json'
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)
df.head()

In [ ]:
df = pd.json_normalize(data)
df.head()

# 🔧 Transformação

In [ ]:
df.info()

## 🔍 Verificação

In [ ]:
for col in df.columns:
  print(f'{col}: {df[col].unique()}')
  if df[col].nunique() < 50:
    print(df[col].unique())
    print('-' * 50)

### 2. Valores Duplicados

In [ ]:
print('Número de Valores Duplicados', df.duplicated().sum())

### 3. Valores Nulos

In [ ]:
print('Número de Valores Nulos \n', df.isnull().sum())

### 4. Valores Vazios ou em Branco

In [ ]:
df.apply(lambda x: x.astype(str).str.strip() == '').sum()

### 5. Lidando com Valores Vazios

In [ ]:
df = df[df['Churn'].astype(str).str.strip() != '']
df = df[df['account.Charges.Total'].astype(str).str.strip() != '']

print(f'Número de linhas após remoção de valores vazios: {len(df)}')

# Verifique novamente os valores vazios para confirmar
print('\nVerificação de valores vazios/em branco após a remoção:')
display(df.apply(lambda x: x.astype(str).str.strip() == '').sum().to_frame(name='Qtd Valores Vazios'))

### 6. Conversão de `account.Charges.Total` para `float`

In [ ]:
df['account.Charges.Total'] = pd.to_numeric(df['account.Charges.Total'])


print('Tipo de dado da coluna account.Charges.Total após conversão:')
df.info()

# 🔧 Padronização dos Dados

## (+) Coluna 'Contas Diárias'

In [ ]:
df['contas_diarias'] = df['account.Charges.Monthly'] / 30.44     # 30.44 dias (média de dias no mês)
df.head()

## Padronização Colunas

In [ ]:
colunas = {                                                                # Dicionário com os novos nomes das colunas.
    'customerID': 'id',
    'customer.gender': 'genero',
    'customer.SeniorCitizen': 'tem +60',
    'customer.Partner': 'possui_parceiro',
    'customer.Dependents': 'possui_dependentes',
    'customer.tenure': 'tempo_contrato',
    'phone.PhoneService': 'servico_telefone',
    'phone.MultipleLines': 'multiplas_linhas',
    'internet.InternetService': 'tipo_internet',
    'internet.OnlineSecurity': 'seguranca_online',
    'internet.OnlineBackup': 'backup_online',
    'internet.DeviceProtection': 'protecao_dispositivo',
    'internet.TechSupport': 'suporte_tecnico',
    'internet.StreamingTV': 'streaming_tv',
    'internet.StreamingMovies': 'streaming_filmes',
    'account.Contract': 'tipo_contrato',
    'account.PaperlessBilling': 'fatura_digital',
    'account.PaymentMethod': 'metodo_pagamento',
    'account.Charges.Monthly': 'valor_mensal',
    'account.Charges.Total': 'total_cobrado'
}

df = df.rename(columns= colunas)
df.head()

## Padronização Linhas

In [ ]:
df['Churn'] = df['Churn'].replace({'No': 'Não', 'Yes': 'Sim'})
df['genero'] = df['genero'].replace({'Female': 'Feminino', 'Male': 'Masculino'})

map_gen = {
    0: 'Não',
    1: 'Sim'
}

df['tem +60'] = df['tem +60'].map(map_gen)
df['possui_parceiro'] = df['possui_parceiro'].replace({'Yes': 'Sim', 'No': 'Não'})
df['possui_dependentes'] = df['possui_dependentes'].replace({'Yes': 'Sim', 'No': 'Não'})
df['servico_telefone'] = df['servico_telefone'].replace({'Yes': 'Sim', 'No': 'Não'})
df['multiplas_linhas'] = df['multiplas_linhas'].replace({'No': 'Não', 'Yes': 'Sim', 'No phone service': 'Sem serviço de telefone'})
df['tipo_internet'] = df['tipo_internet'].replace({'Fiber optic': 'Fibra optica', 'No': 'Não'})
df.head()

In [ ]:
columns_to_translate = ['seguranca_online', 'backup_online', 'protecao_dispositivo', 'suporte_tecnico',
                        'streaming_tv', 'streaming_filmes']
mapping = {'No': 'Não', 'Yes': 'Sim', 'No internet service': 'Sem Serviço de Internet'}
for col in columns_to_translate:
    df[col] = df[col].replace(mapping)

In [ ]:
df['tipo_contrato'] = df['tipo_contrato'].replace({'One year': 'Anual', 'Month-to-month': 'Mensal', 'Two year': 'Bienal'})

In [ ]:
df['fatura_digital'] = df['fatura_digital'].replace({'Yes': 'Sim', 'No': 'Não'})

In [ ]:
df['metodo_pagamento'] = df['metodo_pagamento'].replace({
    'Mailed check': 'Cheque Enviado',
    'Electronic check': 'Cheque eletrônico',
    'Credit card (automatic)': 'Cartão de crédito (automático)',
    'Bank transfer (automatic)': 'Transferência bancária (automática)'
})

In [ ]:
df.head()

# 📊 Carga e análise

## 1. Análise Descritiva

In [ ]:
df.describe()

## 2. Distribuição de Evasão

In [ ]:
import plotly.express as px

In [ ]:
# @title
# Calculate counts and percentages for 'Churn'
churn_counts = df['Churn'].value_counts()
churn_percentages = (df['Churn'].value_counts(normalize=True) * 100).round(1)

# Create the histogram figure
fig = px.histogram(df, x='Churn', color='Churn')

# Update each trace with the correct count and percentage
for trace in fig.data:
    category = trace.x[0] # Get the category from the trace (e.g., 'Não' or 'Sim')
    count = churn_counts.get(category, 0)
    percentage = churn_percentages.get(category, 0.0)
    trace.text = [f"{count} ({percentage:.1f}%)"] # Set text for this trace
    trace.textposition = 'outside' # Position the text outside the bar
    trace.textfont = dict(color='black') # Set text color to black for better visibility

fig.update_layout(hoverlabel=dict(font_color="white"),
                  width=1000,
                  height=600,
                  title='Distribuição de Churn (Permanência vs. Saída)',
                  title_font_size=20,
                  title_x=0.5,
                  xaxis_title='Churn',
                  xaxis=dict(title_font=dict(size=16)),
                  yaxis=dict(title_font=dict(size=16))
)

fig.show()

#### Observação:

*   **Não Churn**: Aproximadamente **73.4%** dos clientes permanecem com a Telecom X.
*   **Churn**: Cerca de **26.6%** dos clientes cancelam o serviço, indicando uma taxa de evasão significativa que precisa de atenção.

## 3. Contagem de Evasão por Variáveis Categóricas

In [ ]:
df.columns

### Análise Demográfica

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Ajuste de Espaçamento Interno (vertical_spacing e horizontal_spacing)
# Quanto menor o valor (ex: 0.05), maior o gráfico fica dentro da área disponível.
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Gênero", "Tem +60 Anos", "Possui Parceiro", "Possui Dependentes"),
    vertical_spacing=0.08,   # Reduzi para os gráficos ficarem mais altos
    horizontal_spacing=0.06  # Reduzi para os gráficos ficarem mais largos
)

cores_map = {'Não': '#636EFA', 'Sim': '#EF553B'}
colunas = ['genero', 'tem +60', 'possui_parceiro', 'possui_dependentes']
posicoes = [(1, 1), (1, 2), (2, 1), (2, 2)]

for col, pos in zip(colunas, posicoes):
    counts = df.groupby([col, 'Churn']).size().reset_index(name='counts')
    counts = counts.sort_values('Churn')

    for status in counts['Churn'].unique():
        data_subset = counts[counts['Churn'] == status]
        fig.add_trace(
            go.Bar(
                name=f"{status}",
                x=data_subset[col],
                y=data_subset['counts'],
                marker_color=cores_map.get(status),
                text=data_subset['counts'],
                textposition='auto',
                texttemplate='%{text}',
                hovertemplate='Status: %{x}<br>Qtd: %{y}'
            ),
            row=pos[0], col=pos[1]
        )

# 2. Ajuste das Margens Externas (margin)
# Definir as margens (l, r, t, b) como valores baixos remove os espaços em branco inúteis.
fig.update_layout(
    hoverlabel=dict(font_color="white"),
    height=600,
    width=1000,
    title_text="Análise Demográfica: Impacto do Churn",
    title_font_size=20,
    title_x=0.5,
    showlegend=True,
    barmode='group',
    margin=dict(l=20, r=20, t=80, b=20), # l=left, r=right, t=top, b=bottom
    legend_title_text="Churn"
)

# Limpeza de legenda repetida
names = set()
fig.for_each_trace(
    lambda trace: trace.update(showlegend=False) if (trace.name in names) else names.add(trace.name)
)
fig.update_traces(textfont_color='white')

fig.show()

#### Observação:

*   **Gênero**: Não há uma diferença notável na taxa de churn entre clientes **Femininos** e **Masculinos**. A proporção de churn é similar para ambos os gêneros.

*   **Clientes com + 60**: Apesar de representarem uma parcela menor da base de clientes, os clientes **+60** exibem uma **taxa de churn significativamente mais alta** em comparação com os **Clientes com - 60.** Este grupo merece atenção especial para estratégias de retenção.

*   **Possui Parceiro**: Clientes que **não possuem parceiro** apresentam uma taxa de churn consideravelmente maior em comparação com aqueles que **possuem parceiro**. Isso sugere que a presença de um parceiro pode indicar maior estabilidade e lealdade ao serviço.

*   **Possui Dependentes**: Clientes que **não possuem dependentes** apresentam uma taxa de churn mais alta em comparação com aqueles que **possuem dependentes**. Isso é consistente com a observação sobre parceiros, sugerindo que laços familiares podem contribuir para uma maior lealdade ao serviço.

### Análise Serviços Infraestrutura

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Configurar as variáveis de Infraestrutura
colunas_infra = ['servico_telefone', 'multiplas_linhas', 'tipo_internet']

# 2. Criar a estrutura (2 linhas, 2 colunas)
# Usamos 'specs' para fazer o gráfico de baixo ocupar as duas colunas (colspan=2)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Serviço de Telefone", "Múltiplas Linhas", "Tipo de Internet"),
    specs=[[{}, {}],           # Primeira linha: dois gráficos normais
           [{"colspan": 2}, None]], # Segunda linha: um gráfico que ocupa 2 espaços e um espaço vazio (None)
    vertical_spacing=0.1,
    horizontal_spacing=0.06
)

cores_map = {'Não': '#636EFA', 'Sim': '#EF553B'}

# 3. Adicionar os gráficos manualmente para garantir as posições
# Gráfico 1 (Posição 1,1)
for status in df['Churn'].unique():
    subset = df[df['Churn'] == status].groupby('servico_telefone').size().reset_index(name='counts')
    fig.add_trace(go.Bar(
        name=f"{status}", x=subset['servico_telefone'], y=subset['counts'],
        marker_color=cores_map.get(status), text=subset['counts'], textposition='auto',
        hovertemplate='Status: %{x}<br>Qtd: %{y}'
    ), row=1, col=1)

# Gráfico 2 (Posição 1,2)
for status in df['Churn'].unique():
    subset = df[df['Churn'] == status].groupby('multiplas_linhas').size().reset_index(name='counts')
    fig.add_trace(go.Bar(
        name=f"{status}", x=subset['multiplas_linhas'], y=subset['counts'],
        marker_color=cores_map.get(status), text=subset['counts'], textposition='auto',
        hovertemplate='Status: %{x}<br>Qtd: %{y}'
    ), row=1, col=2)

# Gráfico 3 (Posição 2,1 - mas com colspan=2 ele se expande)
for status in df['Churn'].unique():
    subset = df[df['Churn'] == status].groupby('tipo_internet').size().reset_index(name='counts')
    fig.add_trace(go.Bar(
        name=f"{status}", x=subset['tipo_internet'], y=subset['counts'],
        marker_color=cores_map.get(status), text=subset['counts'], textposition='auto',
        hovertemplate='Status: %{x}<br>Qtd: %{y}'
    ), row=2, col=1)

# 4. Ajustar o layout para maximizar o tamanho e manter o padrão Plotly
fig.update_layout(
    height=600,
    width=1000,
    title_text="Análise de Serviços de Infraestrutura: Impacto do Churn",
    title_font_size=20,
    title_x=0.5,
    showlegend=True,
    barmode='group',
    margin=dict(l=30, r=30, t=100, b=40),
    legend_title_text="Churn"
)

# Limpeza da legenda repetida
names = set()
fig.for_each_trace(
    lambda trace: trace.update(showlegend=False) if (trace.name in names) else names.add(trace.name)
)

fig.show()

#### Observação:

*   **Serviço de Telefone**: Não parece haver uma diferença significativa na taxa de churn entre clientes que possuem e não possuem o serviço de telefone. A proporção de churn é relativamente similar em ambos os grupos.

*   **Múltiplas Linhas**: Clientes com múltiplas linhas (Sim) e sem múltiplas linhas (Não) apresentam taxas de churn similares. Clientes 'Sem serviço de telefone' naturalmente possuem uma taxa de churn diferente, mas é importante focar na comparação entre 'Sim' e 'Não' para aqueles com o serviço.

*   **Tipo de Internet**: Clientes com serviço de **Fibra Óptica** tendem a ter uma taxa de churn mais alta em comparação com clientes que utilizam **DSL**. Clientes sem serviço de internet ('Não') geralmente têm uma taxa de churn menor, o que pode indicar que esses clientes estão mais focados em outros serviços oferecidos ou têm menos opções de provedores de internet.

### Análise Serviços Agregados

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Variáveis
servicos = [
    'seguranca_online', 'backup_online', 'protecao_dispositivo',
    'suporte_tecnico', 'streaming_tv', 'streaming_filmes'
]

fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=("Segurança Online", "Backup Online", "Proteção de Dispositivo",
                    "Suporte Técnico", "Streaming TV", "Streaming Filmes"),
    vertical_spacing=0.08,
    horizontal_spacing=0.06
)

cores_map = {'Não': '#636EFA', 'Sim': '#EF553B'}

# 2. Loop com limpeza profunda de strings
for i, col in enumerate(servicos):
    row = (i // 2) + 1
    col_pos = (i % 2) + 1

    # Criamos a cópia e limpamos TUDO: espaços extras e forçamos a troca
    df_temp = df.copy()

    # Converte para string e remove espaços no início/fim
    df_temp[col] = df_temp[col].astype(str).str.strip()

    # Mapeamento direto (ajuste o lado esquerdo se o seu texto for diferente)
    # Aqui cobrimos as versões mais comuns do dataset da IBM/Kaggle
    mapeamento = {
        'Sem Serviço de Internet': 'S/ Internet',

    }

    df_temp[col] = df_temp[col].replace(mapeamento)

    # Se ainda assim sobrar algo longo que contenha "internet", encurtamos via lambda
    df_temp[col] = df_temp[col].apply(lambda x: 'S/ Internet' if 'internet' in x.lower() else x)

    counts = df_temp.groupby([col, 'Churn']).size().reset_index(name='counts')
    counts = counts.sort_values('Churn')

    for status in counts['Churn'].unique():
        data_subset = counts[counts['Churn'] == status]

        fig.add_trace(
            go.Bar(
                name=f"{status}",
                x=data_subset[col],
                y=data_subset['counts'],
                marker_color=cores_map.get(status),
                text=data_subset['counts'],
                textposition='auto',
                texttemplate='%{text}',
                hovertemplate='Status: %{x}<br>Qtd: %{y}'
            ),
            row=row, col=col_pos
        )

# 3. Layout e Ajustes de Eixo
fig.update_layout(
    height=800,
    width=1000,
    title_text="Análise de Serviços Agregados: Impacto do Churn",
    title_font_size=20,
    title_x=0.5,
    showlegend=True,
    barmode='group',
    margin=dict(l=30, r=30, t=100, b=40),
    legend_title_text="Churn"
)

# Força o texto na horizontal
fig.update_xaxes(tickangle=0)

# Limpeza da legenda
names = set()
fig.for_each_trace(
    lambda trace: trace.update(showlegend=False) if (trace.name in names) else names.add(trace.name)
)

fig.show()

#### Observação:

*   **Segurança Online**: Clientes que não utilizam o serviço de segurança online ('Não') apresentam uma taxa de churn significativamente mais alta em comparação com aqueles que o utilizam ('Sim') ou que não possuem serviço de internet.

*   **Backup Online**: Clientes que não utilizam o serviço de backup online ('Não') tendem a ter uma taxa de churn mais elevada em comparação com aqueles que utilizam ('Sim') ou que não possuem serviço de internet. Isso sugere a importância de serviços adicionais na retenção.

*   **Proteção de Dispositivo**: Similar aos serviços de segurança e backup online, clientes que **não** possuem proteção de dispositivo ('Não') tendem a apresentar uma taxa de churn mais elevada em comparação com aqueles que utilizam o serviço ('Sim') ou que não possuem serviço de internet.

*   **Suporte Técnico**: Clientes que não utilizam o serviço de suporte técnico ('Não') apresentam uma taxa de churn significativamente mais alta em comparação com aqueles que utilizam ('Sim') ou que não possuem serviço de internet. Isso reforça a ideia de que serviços adicionais de valor agregado são importantes para a retenção.

*   **Streaming TV**: Clientes que **não** utilizam o serviço de streaming de TV ('Não') apresentam uma taxa de churn mais alta em comparação com aqueles que o utilizam ('Sim'). Clientes sem serviço de internet tendem a ter uma taxa de churn menor, o que pode sugerir que o streaming de TV, quando presente, se torna um fator na decisão de churn.

*   **Streaming Filmes**: Similar ao Streaming TV, clientes que **não** utilizam o serviço de streaming de filmes ('Não') apresentam uma taxa de churn mais alta em comparação com aqueles que o utilizam ('Sim'). Clientes sem serviço de internet tendem a ter uma taxa de churn menor, sugerindo que o serviço de streaming de filmes, quando presente, influencia a decisão de churn.

### Análise Contratual e Financeira

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1. Configurar as variáveis de Infraestrutura
colunas_infra = ['tipo_contrato', 'fatura_digital', 'metodo_pagamento']

# 2. Criar a estrutura (2 linhas, 2 colunas)
# Usamos 'specs' para fazer o gráfico de baixo ocupar as duas colunas (colspan=2)
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Tipo de Contrato", "Fatura Digital", "Método de Pagamento"),
    specs=[[{}, {}],           # Primeira linha: dois gráficos normais
           [{"colspan": 2}, None]], # Segunda linha: um gráfico que ocupa 2 espaços e um espaço vazio (None)
    vertical_spacing=0.1,
    horizontal_spacing=0.06
)

cores_map = {'Não': '#636EFA', 'Sim': '#EF553B'}

# 3. Adicionar os gráficos manualmente para garantir as posições
# Gráfico 1 (Posição 1,1)
for status in df['Churn'].unique():
    subset = df[df['Churn'] == status].groupby('tipo_contrato').size().reset_index(name='counts')
    fig.add_trace(go.Bar(
        name=f"{status}", x=subset['tipo_contrato'], y=subset['counts'],
        marker_color=cores_map.get(status), text=subset['counts'], textposition='auto',
        hovertemplate='Status: %{x}<br>Qtd: %{y}'
    ), row=1, col=1)

# Gráfico 2 (Posição 1,2)
for status in df['Churn'].unique():
    subset = df[df['Churn'] == status].groupby('fatura_digital').size().reset_index(name='counts')
    fig.add_trace(go.Bar(
        name=f"{status}", x=subset['fatura_digital'], y=subset['counts'],
        marker_color=cores_map.get(status), text=subset['counts'], textposition='auto',
        hovertemplate='Status: %{x}<br>Qtd: %{y}'
    ), row=1, col=2)

# Gráfico 3 (Posição 2,1 - mas com colspan=2 ele se expande)
for status in df['Churn'].unique():
    subset = df[df['Churn'] == status].groupby('metodo_pagamento').size().reset_index(name='counts')
    fig.add_trace(go.Bar(
        name=f"{status}", x=subset['metodo_pagamento'], y=subset['counts'],
        marker_color=cores_map.get(status), text=subset['counts'], textposition='auto',
        hovertemplate='Status: %{x}<br>Qtd: %{y}'
    ), row=2, col=1)

# 4. Ajustar o layout para maximizar o tamanho e manter o padrão Plotly
fig.update_layout(
    height=600,
    width=1000,
    title_text="Análise Contratual e Financeira: Impacto do Churn",
    title_font_size=20,
    title_x=0.5,
    showlegend=True,
    barmode='group',
    margin=dict(l=30, r=30, t=100, b=40),
    legend_title_text="Churn"
)

# Limpeza da legenda repetida
names = set()
fig.for_each_trace(
    lambda trace: trace.update(showlegend=False) if (trace.name in names) else names.add(trace.name)
)

fig.show()

#### Observação:

*   **Tipo de Contrato**: Clientes com contrato **Mensal** apresentam uma taxa de churn drasticamente mais alta em comparação com clientes com contratos **Anual** e **Bienal**. Contratos de longa duração ('Anual' e 'Bienal') demonstram maior lealdade, sendo o contrato 'Bienal' o que possui a menor taxa de evasão.

*   **Fatura Digital**: Clientes que optam pela **fatura digital ('Sim')** apresentam uma taxa de churn mais elevada em comparação com aqueles que recebem a fatura impressa ('Não'). Isso pode indicar que a fatura digital, embora prática, pode estar associada a uma menor interação ou vínculo com a empresa.

*   **Método de Pagamento**: Clientes que utilizam o **Cheque eletrônico** como método de pagamento apresentam uma taxa de churn significativamente mais alta em comparação com outros métodos, como Cartão de crédito (automático) e Transferência bancária (automática). Isso sugere que os clientes que preferem o cheque eletrônico podem ter menos vínculo com a empresa ou encontrar mais atrito no processo de pagamento, levando a uma maior propensão ao cancelamento.

## 4. Contagem de Evasão por Variáveis Numéricas

### Análise Tenure e Custos

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# 1. Garantir que as variáveis estão no formato correto
for col in ['tempo_contrato', 'valor_mensal']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. Criar a estrutura (2 linhas, 2 colunas)
fig = make_subplots(
    rows=2, cols=2, # Corrected: 2 rows for 2 variables, each with a histogram and boxplot
    subplot_titles=(
        "Distribuição de Tenure (Meses)", "Boxplot de Tenure (Meses)",
        "Distribuição de Valor Mensal ($)", "Boxplot de Valor Mensal ($)"
    ),
    horizontal_spacing=0.1,
    vertical_spacing=0.18
)

cores = {'Não': '#636EFA', 'Sim': '#EF553B'}
vars_num = ['tempo_contrato', 'valor_mensal']

# 3. Loop para preencher a grade
for i, col in enumerate(vars_num):
    row_index = i + 1 # row_index will be 1 for tempo_contrato, 2 for valor_mensal

    for status in ['Não', 'Sim']:
        # Histograma (Coluna 1)
        fig.add_trace(
            go.Histogram(
                x=df[df['Churn'] == status][col],
                name=f"{status}", # Keep the name for legend clarity
                marker_color=cores[status],
                opacity=0.7,
                showlegend=True if i == 0 else False, # Show legend only for the first variable
                hovertemplate=f"Churn: {status}<br>{col}: %{{x}}<br>Contagem: %{{y}}<extra></extra>"
            ), row=row_index, col=1
        )

        # Boxplot (Coluna 2)
        # Calculate min, max, mean for the current group and column
        data_for_boxplot = df[df['Churn'] == status][col].dropna()
        min_val = data_for_boxplot.min()
        max_val = data_for_boxplot.max()
        mean_val = data_for_boxplot.mean()

        fig.add_trace(
            go.Box(
                y=data_for_boxplot,
                name=f"{status}", # Keep the name for legend clarity
                marker_color=cores[status],
                boxmean=True,
                showlegend=False, # Don't show legend for boxplots (already shown by histograms)
                hovertemplate=f"Churn: {status}<br>{col}: %{{y}}<br>Min: {min_val:.2f}<br>Max: {max_val:.2f}<br>Mean: {mean_val:.2f}<extra></extra>"
            ), row=row_index, col=2
        )

# 4. Ajustes de Layout
fig.update_layout(
    height=600, # Adjusted height for better spacing
    width=1000,
    title_text="Análise de Tenure e Custos: Impacto do Churn",
    title_font_size=20,
    title_x=0.5,
    barmode='overlay',
    template='plotly',
    margin=dict(l=30, r=30, t=100, b=40),
    legend_title_text="Churn" # Added a title for the legend
)

# 5. Adicionar títulos aos eixos Y para cada subplot
fig.update_yaxes(title_text="Contagem de Clientes", row=1, col=1)
fig.update_yaxes(title_text="Tempo de Contrato (Meses)", row=1, col=2)
fig.update_yaxes(title_text="Contagem de Clientes", row=2, col=1)
fig.update_yaxes(title_text="Valor Mensal ($)", row=2, col=2)

# 6. Adicionar títulos aos eixos X para cada subplot (Histograms only, boxplot X-axis is categorical)
fig.update_xaxes(title_text="Tempo de Contrato (Meses)", row=1, col=1)
fig.update_xaxes(title_text="Valor Mensal ($)", row=2, col=1)


fig.show()

#### Observação:

*   **Tempo de Contrato (Tenure)**: Clientes com menos tempo de contrato têm uma taxa de Churn significativamente mais alta, enquanto a lealdade aumenta com a duração do contrato.

*   **Valor Mensal**: Clientes com valores mensais mais altos mostram maior propensão ao Churn, indicando que o custo percebido é um fator crítico.

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# 1. Garantir que as variáveis estão no formato correto
for col in ['total_cobrado', 'contas_diarias']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 2. Criar a estrutura (2 linhas, 2 colunas)
fig = make_subplots(
    rows=2, cols=2, # Corrected: 2 rows for 2 variables, each with a histogram and boxplot
    subplot_titles=(
        "Distribuição: Cobrança Total ($)", "Estatística: Cobrança Total ($)",
        "Distribuição: Contas Diárias ($)", "Estatística: Contas Diárias($)"
    ),
    horizontal_spacing=0.1,
    vertical_spacing=0.18
)

cores = {'Não': '#636EFA', 'Sim': '#EF553B'}
vars_num = ['total_cobrado', 'contas_diarias']

# 3. Loop para preencher a grade
for i, col in enumerate(vars_num):
    row_index = i + 1 # row_index will be 1 for total_cobrado, 2 for contas_diarias

    for status in ['Não', 'Sim']:
        # Histograma (Coluna 1)
        fig.add_trace(
            go.Histogram(
                x=df[df['Churn'] == status][col],
                name=f"{status}", # Keep the name for legend clarity
                marker_color=cores[status],
                opacity=0.7,
                showlegend=True if i == 0 else False, # Show legend only for the first variable
                hovertemplate=f"Churn: {status}<br>{col}: %{{x}}<br>Contagem: %{{y}}<extra></extra>"
            ), row=row_index, col=1
        )

        # Boxplot (Coluna 2)
        fig.add_trace(
            go.Box(
                y=df[df['Churn'] == status][col],
                name=f"{status}", # Keep the name for legend clarity
                marker_color=cores[status],
                boxmean=True,
                showlegend=False, # Don't show legend for boxplots (already shown by histograms)
                hovertemplate=f"Churn: {status}<br>{col}: %{{y}}<extra></extra>"
            ), row=row_index, col=2
        )

# 4. Ajustes de Layout
fig.update_layout(
    height=600, # Adjusted height for better spacing
    width=1000,
    title_text="Análise de Tenure e Custos: Impacto do Churn",
    title_font_size=20,
    title_x=0.5,
    barmode='overlay',
    template='plotly',
    margin=dict(l=30, r=30, t=100, b=40),
    legend_title_text="Churn" # Added a title for the legend
)

# 5. Adicionar títulos aos eixos Y para cada subplot
fig.update_yaxes(title_text="Contagem de Clientes", row=1, col=1)
fig.update_yaxes(title_text="Cobrança Total ($)", row=1, col=2)
fig.update_yaxes(title_text="Contagem de Clientes", row=2, col=1)
fig.update_yaxes(title_text="Valor Diário ($)", row=2, col=2)

# 6. Adicionar títulos aos eixos X para cada subplot (Histograms only, boxplot X-axis is categorical)
fig.update_xaxes(title_text="Cobrança Total ($)", row=1, col=1)
fig.update_xaxes(title_text="Valor Diário ($)", row=2, col=1)
# For boxplots, the x-axis typically represents categories, which are implied by the plot structure, so no explicit numerical x-axis title needed.

fig.show()

#### Observação:

*   **Cobrança Total**: Clientes com menor cobrança total acumulada ('total_cobrado') apresentam maior taxa de Churn, indicando que clientes com menor histórico de gastos são mais propensos a cancelar.

*   **Contas Diárias**: A taxa de Churn tende a aumentar com o valor das 'contas_diarias', sugerindo que um custo diário percebido mais alto está associado a uma maior propensão à evasão.

In [ ]:
# @title
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# 1. Preparação e Transformação
df_plot = df.copy()
df_plot['Churn_Num'] = df_plot['Churn'].map({'Sim': 1, 'Não': 0})

# Criando as variáveis numéricas corretamente
df_plot['valor_mensal'] = pd.to_numeric(df_plot['valor_mensal'], errors='coerce')
df_plot['contas_diarias'] = df_plot['valor_mensal'] / 30.44
df_plot['total_cobrado'] = pd.to_numeric(df_plot['total_cobrado'], errors='coerce')
df_plot['tempo_contrato'] = pd.to_numeric(df_plot['tempo_contrato'], errors='coerce')

# 2. Função para calcular taxa por faixas com Eixo X dinâmico
def obter_taxa_por_faixa(df_f, coluna, bins=10):
    # Remove valores nulos para o cálculo de faixas
    df_limpo = df_f.dropna(subset=[coluna])
    # Cria as faixas baseadas nos valores REAIS da coluna
    df_limpo['faixa'] = pd.cut(df_limpo[coluna], bins=bins)
    # Calcula a média
    resumo = df_limpo.groupby('faixa', observed=True)['Churn_Num'].mean() * 100
    # Eixo X recebe o ponto médio de cada intervalo real
    x_values = [intervalo.mid for intervalo in resumo.index]
    return x_values, resumo.values

# 3. Estrutura da Figura
fig_linhas = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Taxa de Churn por Tenure (Meses)", "Taxa de Churn por Valor Mensal ($)",
        "Taxa de Churn por Cobrança Total ($)", "Taxa de Churn por Contas Diárias ($)"
    ),
    vertical_spacing=0.15,
    horizontal_spacing=0.06
)

vars_num = ['tempo_contrato', 'valor_mensal', 'total_cobrado', 'contas_diarias']

# 4. Loop de preenchimento
for i, col in enumerate(vars_num):
    row = (i // 2) + 1
    col_pos = (i % 2) + 1

    if col == 'tempo_contrato':
        resumo = df_plot.groupby(col)['Churn_Num'].mean() * 100
        x, y = resumo.index, resumo.values
    else:
        x, y = obter_taxa_por_faixa(df_plot, col)

    fig_linhas.add_trace(
        go.Scatter(
            x=x, y=y,
            mode='lines+markers',
            line=dict(color='#636EFA', width=2),
            marker=dict(size=5),
            hovertemplate="Valor: %{x:.2f}<br>Taxa: %{y:.2f}%<extra></extra>"
        ), row=row, col=col_pos
    )

# 5. Ajustes de Layout e Eixos Independentes
fig_linhas.update_layout(
    hoverlabel=dict(font_color="white"),
    height=800, width=1000,
    title_text="Tendência da Taxa de Churn: Tenure e Custos",
    title_font_size=20,
    title_x=0.5,
    template='plotly',
    showlegend=False,
    margin=dict(l=30, r=30, t=100, b=40)
)

# Aplicando configurações de eixos
fig_linhas.update_yaxes(ticksuffix="%")
# Configurei para 'matches=None' para garantir que cada X tenha sua própria escala
fig_linhas.update_xaxes(matches=None)

fig_linhas.show()

#### Observação:

A taxa de Churn está inversamente ligada ao tempo de contrato e ao valor total cobrado: clientes com menor tempo de permanência e menor gasto acumulado são mais propensos a cancelar. Em contrapartida, o Churn tende a aumentar com o valor mensal e diário percebido do serviço, especialmente para valores mais altos.

# Extra: Análise de Correlação entre Variáveis

In [ ]:
df.info()

In [ ]:
df_ml = df.copy()

## Transformação dos Dados

In [ ]:
X = df_ml.drop(['id', 'Churn'], axis=1)
y = df_ml['Churn']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
colunas = X.columns

In [ ]:
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'),
    ['genero', 'tem +60', 'possui_parceiro', 'possui_dependentes', 'servico_telefone',
     'multiplas_linhas', 'tipo_internet', 'seguranca_online', 'backup_online', 'protecao_dispositivo',
     'suporte_tecnico', 'streaming_tv', 'streaming_filmes', 'tipo_contrato', 'fatura_digital', 'metodo_pagamento']),
                                  remainder='passthrough',
                                  sparse_threshold=0)

In [ ]:
X = one_hot.fit_transform(X)

In [ ]:
one_hot.get_feature_names_out(colunas)

In [ ]:
new_column_names = one_hot.get_feature_names_out(colunas)

def clean_column_name(col_name):
    if col_name.startswith('onehotencoder__'):
        return col_name.replace('onehotencoder__', '')
    elif col_name.startswith('remainder__'):
        return col_name.replace('remainder__', '')
    return col_name

cleaned_column_names = [clean_column_name(name) for name in new_column_names]
X_encoded = pd.DataFrame(X, columns=cleaned_column_names)
X_encoded.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
y = label_encoder.fit_transform(y)

In [ ]:
y

## Correlação dos Dados

In [ ]:
# @title
df_encoded = X_encoded.copy()
df_encoded['Churn'] = y

# Calculate the correlation of all features with the 'Churn' variable
correlation_with_all = df_encoded.corr(numeric_only=True)['Churn'].sort_values(ascending=False)
print(correlation_with_all)

#### Observação:

*   A correlação com a variável 'Churn' revela quais características têm maior impacto na evasão de clientes. Valores positivos indicam que, à medida que a característica aumenta, o churn também tende a aumentar. Valores negativos indicam o oposto.
*   Por exemplo, 'tempo_contrato' e 'total_cobrado' apresentam correlações negativas, sugerindo que clientes com maior tempo de contrato e maior valor total pago são menos propensos a cancelar. Em contraste, 'valor_mensal' e 'contas_diarias' têm correlações positivas, indicando que custos mensais e diários mais altos estão associados a maior churn.

In [ ]:
# @title
df_encoded = X_encoded.copy()
df_encoded['Churn'] = y

# Calculate the correlation of all features with the 'Churn' variable
correlation_with_all = df_encoded.corr(numeric_only=True)['Churn'].sort_values(ascending=False)

# Obter Correlação
feminino_correlation = -correlation_with_all.loc['genero_Masculino']
senior_correlation = -correlation_with_all.loc['tem +60_Sim']
parceiro_correlation = -correlation_with_all.loc['possui_parceiro_Sim']
dependente_correlation = correlation_with_all.loc['possui_dependentes_Sim']

# Filtrar as correlações para as variáveis demográficas
service_correlations = pd.Series({
    'genero_Feminino': feminino_correlation,
    'genero_Masculino': correlation_with_all.loc['genero_Masculino'],
    'tem +60_Sim': correlation_with_all.loc['tem +60_Sim'],
    'tem +60_Nao': senior_correlation,
    'possui_parceiro_Sim': correlation_with_all.loc['possui_parceiro_Sim'],
    'possui_parceiro_Nao': parceiro_correlation,
    'possui_dependentes_Sim': correlation_with_all.loc['possui_dependentes_Sim'],
    'possui_dependentes_Não': dependente_correlation
})

# Renomear as chaves para melhor visualização no gráfico
service_correlations = service_correlations.rename(index={
    'genero_Feminino': 'Feminino',
    'genero_Masculino': 'Masculino',
    'tem +60_Sim': 'Tem +60',
    'tem +60_Nao': 'Tem -60',
    'possui_parceiro_Sim': 'Parceiro(a)',
    'possui_parceiro_Nao': 'Solteiro(a)',
    'possui_dependentes_Sim': 'Dependentes',
    'possui_dependentes_Não': 'S/ Dependentes'

})

custom_rb_weaker_red = [
    [0.0, 'rgb(99, 102, 255)'],    # Azul escuro (como o RdBu_r para valores negativos)
    [0.5, 'rgb(210, 210, 210)'], # Branco (para valores próximos de zero)
    [1.0, 'rgb(239, 85, 59)']  # Vermelho mais fraco (para valores positivos)
]


# Criar um DataFrame para o Plotly Express
df_service_corr = service_correlations.reset_index()
df_service_corr.columns = ['Variáveis Demográficas', 'Churn']

# Criar o gráfico de barras
fig = px.bar(df_service_corr, x='Variáveis Demográficas', y='Churn',
             title='Correlação das Variáveis Demográficas com Churn',
             color='Churn', # Cores baseadas no valor da correlação
             color_continuous_scale=custom_rb_weaker_red,
             range_color=[-0.3, 0.3],
             text_auto=".2f") # Adiciona o valor da correlação nas barras

fig.update_layout(hoverlabel=dict(font_color="white"),
                  width=1000,
                  height=600,
                  title_font_size=20,
                  title_x=0.5)

# Mover as configurações de fonte para os eixos
fig.update_xaxes(title_font=dict(size=16))
fig.update_yaxes(title_font=dict(size=16))

fig.update_yaxes(rangemode='tozero') # Começar o eixo Y em zero

fig.show()

#### Observação Demográfica (Correlação com Churn):

*   **Gênero**: A correlação com o Churn é quase nula, indicando que o gênero não é um fator significativo para a evasão de clientes.

*   **Clientes com + 60**: Apresentam uma correlação positiva moderada com o Churn, sugerindo que clientes idosos têm maior propensão a cancelar.

*   **Possui Parceiro**: Ter um parceiro tem uma correlação negativa moderada com o Churn, indicando que esses clientes são menos propensos a evadir. Clientes sem parceiro mostram o oposto.

*   **Possui Dependentes**: Similar ao parceiro, ter dependentes também tem uma correlação negativa com o Churn, significando que clientes com dependentes são mais leais.

In [ ]:
# @title
# Obter Correlação
servico_tel_correlation = -correlation_with_all.loc['servico_telefone_Sim']


# Filtrar as correlações para as Serviços Infraestrutura
service_correlations = pd.Series({
    'servico_telefone_Sim': correlation_with_all.loc['servico_telefone_Sim'],
    'servico_telefone_Nao': servico_tel_correlation,
    'multiplas_linhas_Sim': correlation_with_all.loc['multiplas_linhas_Sim'],
    'multiplas_linhas_Não': correlation_with_all.loc['multiplas_linhas_Não'],
    'multiplas_linhas_Sem serviço de telefone': correlation_with_all.loc['multiplas_linhas_Sem serviço de telefone'],
    'tipo_internet_Não': correlation_with_all.loc['tipo_internet_Não'],
    'tipo_internet_DSL': correlation_with_all.loc['tipo_internet_DSL'],
    'tipo_internet_Fibra optica': correlation_with_all.loc['tipo_internet_Fibra optica']

})

# Renomear as chaves para melhor visualização no gráfico
service_correlations = service_correlations.rename(index={
    'servico_telefone_Sim': 'Servico telefone',
    'servico_telefone_Nao': 'S/ Servico telefone',
    'multiplas_linhas_Sim': 'Multi linhas',
    'multiplas_linhas_Não': 'S/ Multi linhas',
    'multiplas_linhas_Sem serviço de telefone': 'Multi linhas S/ Serviço',
    'tipo_internet_Não': 'S/ Internet',
    'tipo_internet_DSL': 'DSL',
    'tipo_internet_Fibra optica': 'Fibra Optica'

})

custom_rb_weaker_red = [
    [0.0, 'rgb(99, 102, 255)'],    # Azul escuro (como o RdBu_r para valores negativos)
    [0.5, 'rgb(210, 210, 210)'], # Branco (para valores próximos de zero)
    [1.0, 'rgb(239, 85, 59)']  # Vermelho mais fraco (para valores positivos)
]


# Criar um DataFrame para o Plotly Express
df_service_corr = service_correlations.reset_index()
df_service_corr.columns = ['Serviços Infraestrutura', 'Churn']

# Criar o gráfico de barras
fig = px.bar(df_service_corr, x='Serviços Infraestrutura', y='Churn',
             title='Correlação Serviços de Infraestrutura com Churn',
             color='Churn', # Cores baseadas no valor da correlação
             color_continuous_scale=custom_rb_weaker_red,
             range_color=[-0.3, 0.3], # Ajustar a escala de cores para melhor contraste
             text_auto=".2f") # Adiciona o valor da correlação nas barras

fig.update_layout(hoverlabel=dict(font_color="white"),
                  width=1000,
                  height=600,
                  title_font_size=20,
                  title_x=0.5)

# Mover as configurações de fonte para os eixos
fig.update_xaxes(title_font=dict(size=16))
fig.update_yaxes(title_font=dict(size=16))

fig.update_yaxes(rangemode='tozero') # Começar o eixo Y em zero

fig.show()

#### Observação Serviços de Infraestrutura (Correlação com Churn):

*   **Serviço de Telefone e Múltiplas Linhas**: Ambos apresentam correlações muito baixas com o Churn, indicando que a presença ou ausência desses serviços tem pouco impacto na decisão de cancelamento do cliente.

*   **Tipo de Internet**: Clientes com serviço de **Fibra Óptica** mostram uma correlação positiva moderada com o Churn, sugerindo que são mais propensos a cancelar. Em contraste, clientes **sem serviço de Internet** e com **DSL** têm correlações negativas, indicando menor propensão à evasão.

In [ ]:
# @title
# Filtrar as correlações para os serviços com valor agregado
service_correlations = correlation_with_all.filter(items=[
    'seguranca_online_Sim',
    'backup_online_Sim',
    'protecao_dispositivo_Sim',
    'suporte_tecnico_Sim',
    'streaming_tv_Sim',
    'streaming_filmes_Sim'
])

# Renomear as chaves para melhor visualização no gráfico
service_correlations = service_correlations.rename(index={
    'seguranca_online_Sim': 'Segurança Online',
    'backup_online_Sim': 'Backup Online',
    'protecao_dispositivo_Sim': 'Proteção Dispositivo',
    'suporte_tecnico_Sim': 'Suporte Técnico',
    'streaming_tv_Sim': 'Streaming TV',
    'streaming_filmes_Sim': 'Streaming Filmes'
})

# Criar um DataFrame para o Plotly Express
df_service_corr = service_correlations.reset_index()
df_service_corr.columns = ['Serviço Agregados', 'Churn']


custom_rb_weaker_red = [
    [0.0, 'rgb(99, 102, 255)'],    # Azul escuro (como o RdBu_r para valores negativos)
    [0.5, 'rgb(210, 210, 210)'], # Branco (para valores próximos de zero)
    [1.0, 'rgb(239, 85, 59)']  # Vermelho mais fraco (para valores positivos)
]

# Criar o gráfico de barras
fig = px.bar(df_service_corr, x='Serviço Agregados', y='Churn',
             title='Correlação dos Serviços Agregados com Churn',
             color='Churn', # Cores baseadas no valor da correlação
             color_continuous_scale=custom_rb_weaker_red,
             range_color=[-0.3, 0.3], # Ajustar a escala de cores para melhor contraste
             text_auto=".2f") # Adiciona o valor da correlação nas barras

fig.update_layout(hoverlabel=dict(font_color="white"),
                  width=1000,
                  height=600,
                  title_font_size=20,
                  title_x=0.5)

fig.update_yaxes(rangemode='tozero') # Começar o eixo Y em zero

fig.show()

#### Observação Serviços Agregados (Correlação com Churn):

*   **Serviços com correlação negativa (menos Churn)**: A presença de serviços como **Segurança Online**, **Suporte Técnico**, **Backup Online** e **Proteção de Dispositivo** tem correlação negativa com o Churn, indicando que clientes com esses serviços são menos propensos a cancelar. Isso sugere que esses adicionais aumentam a satisfação e lealdade.

*   **Serviços com correlação positiva (mais Churn)**: A presença de **Streaming TV** e **Streaming Filmes** apresenta uma correlação positiva, embora mais fraca. Isso pode indicar que, para clientes que valorizam esses serviços, a expectativa é maior, e qualquer insatisfação pode levar ao Churn.

In [ ]:
# @title
# Filtrar as correlações para a ausência de serviços com valor agregado
absence_service_correlations = correlation_with_all.filter(items=[
    'seguranca_online_Não',
    'seguranca_online_Sem Serviço de Internet',
    'backup_online_Não',
    'backup_online_Sem Serviço de Internet',
    'protecao_dispositivo_Não',
    'protecao_dispositivo_Sem Serviço de Internet',
    'suporte_tecnico_Não',
    'suporte_tecnico_Sem Serviço de Internet',
    'streaming_tv_Não',
    'streaming_tv_Sem Serviço de Internet',
    'streaming_filmes_Não',
    'streaming_filmes_Sem Serviço de Internet'
])

# Renomear as chaves para melhor visualização no gráfico
absence_service_correlations = absence_service_correlations.rename(index={
    'seguranca_online_Não': 'Sem Segurança Online',
    'seguranca_online_Sem Serviço de Internet': 'Segurança Online<br> S/ Internet',
    'backup_online_Não': 'Sem Backup Online',
    'backup_online_Sem Serviço de Internet': 'Backup Online<br> S/ Internet',
    'protecao_dispositivo_Não': 'Sem Proteção Dispositivo',
    'protecao_dispositivo_Sem Serviço de Internet': 'Proteção Dispositivo<br> S/ Internet',
    'suporte_tecnico_Não': 'Sem Suporte Técnico',
    'suporte_tecnico_Sem Serviço de Internet': 'Suporte Técnico<br> S/ Internet',
    'streaming_tv_Não': 'Sem Streaming TV',
    'streaming_tv_Sem Serviço de Internet': 'Streaming TV<br> S/ Internet',
    'streaming_filmes_Não': 'Sem Streaming Filmes',
    'streaming_filmes_Sem Serviço de Internet': 'Streaming Filmes<br> S/ Internet'
})

# Criar um DataFrame para o Plotly Express
df_absence_service_corr = absence_service_correlations.reset_index()
df_absence_service_corr.columns = ['Ausência Serviços Agregados', 'Churn']


custom_rb_weaker_red = [
    [0.0, 'rgb(99, 102, 255)'],    # Azul escuro (como o RdBu_r para valores negativos)
    [0.5, 'rgb(210, 210, 210)'], # Branco (para valores próximos de zero)
    [1.0, 'rgb(239, 85, 59)']  # Vermelho mais fraco (para valores positivos)
]

# Criar o gráfico de barras
fig = px.bar(df_absence_service_corr, x='Ausência Serviços Agregados', y='Churn',
             title='Correlação da Ausência de Serviços Agregados com Churn',
             color='Churn', # Cores baseadas no valor da correlação
             color_continuous_scale=custom_rb_weaker_red,
             range_color=[-0.3, 0.3], # Ajustar a escala de cores para melhor contraste
             text_auto=".2f") # Adiciona o valor da correlação nas barras


fig.update_layout(hoverlabel=dict(font_color="white"),
                  title_font_size=20,
                  width=1000,
                  height=600,
                  title_x=0.5,
                  )

fig.update_yaxes(rangemode='tozero') # Começar o eixo Y em zero

fig.show()

#### Observação Ausência de Serviços Agregados (Correlação com Churn):

*   **Serviços com correlação positiva (mais Churn)**: A ausência de serviços como **Segurança Online**, **Suporte Técnico**, **Backup Online** e **Proteção de Dispositivo** tem uma correlação positiva forte com o Churn, indicando que clientes sem esses serviços são mais propensos a cancelar. Isso sugere que a falta desses adicionais leva a uma menor satisfação e maior evasão.

*   **Serviços de Streaming (Sem Streaming TV / Sem Streaming Filmes)**: A ausência de serviços de streaming também apresenta correlação positiva com o Churn, embora mais fraca. Isso pode indicar que, para alguns clientes, a falta desses serviços pode ser um fator na decisão de churn.

In [ ]:
# @title
# Obter Correlação
fatura_digital_correlation = -correlation_with_all.loc['fatura_digital_Sim']


# Filtrar as correlações para as Serviços Infraestrutura
service_correlations = pd.Series({
    'tipo_contrato_Mensal': correlation_with_all.loc['tipo_contrato_Mensal'],
    'tipo_contrato_Anual': correlation_with_all.loc['tipo_contrato_Anual'],
    'tipo_contrato_Bienal': correlation_with_all.loc['tipo_contrato_Bienal'],
    'fatura_digital_Sim': correlation_with_all.loc['fatura_digital_Sim'],
    'fatura_digital_Não': fatura_digital_correlation,
    'metodo_pagamento_Cheque eletrônico': correlation_with_all.loc['metodo_pagamento_Cheque eletrônico'],
    'metodo_pagamento_Cheque Enviado': correlation_with_all.loc['metodo_pagamento_Cheque Enviado'],
    'metodo_pagamento_Transferência bancária (automática)': correlation_with_all.loc['metodo_pagamento_Transferência bancária (automática)'],
    'metodo_pagamento_Cartão de crédito (automático)': correlation_with_all.loc['metodo_pagamento_Cartão de crédito (automático)']

})

# Renomear as chaves para melhor visualização no gráfico
service_correlations = service_correlations.rename(index={
    'tipo_contrato_Mensal': 'Contrato Mensal',
    'tipo_contrato_Anual': 'Contrato Anual',
    'tipo_contrato_Bienal': 'Contrato Bienal',
    'fatura_digital_Sim': 'Fatura Digital',
    'fatura_digital_Não': 'S/ Fatura Digital',
    'metodo_pagamento_Cheque eletrônico': 'Cheque eletrônico',
    'metodo_pagamento_Cheque Enviado': 'Cheque Enviado',
    'metodo_pagamento_Transferência bancária (automática)': 'Transferência bancária <br> (automática)',
    'metodo_pagamento_Cartão de crédito (automático)':'Cartão de crédito <br> (automático)'

})

custom_rb_weaker_red = [
    [0.0, 'rgb(99, 102, 255)'],    # Azul escuro (como o RdBu_r para valores negativos)
    [0.5, 'rgb(210, 210, 210)'], # Branco (para valores próximos de zero)
    [1.0, 'rgb(239, 85, 59)']  # Vermelho mais fraco (para valores positivos)
]


# Criar um DataFrame para o Plotly Express
df_service_corr = service_correlations.reset_index()
df_service_corr.columns = ['Variáveis Contratual/Financeira', 'Churn']

# Criar o gráfico de barras
fig = px.bar(df_service_corr, x='Variáveis Contratual/Financeira', y='Churn',
             title='Correlação Variáveis Contratual/Financeira com Churn',
             color='Churn', # Cores baseadas no valor da correlação
             color_continuous_scale=custom_rb_weaker_red,
             range_color=[-0.3, 0.3], # Ajustar a escala de cores para melhor contraste
             text_auto=".2f") # Adiciona o valor da correlação nas barras


fig.update_layout(hoverlabel=dict(font_color="white"),
                  width=1000,
                  height=600,
                  title_font_size=20,
                  title_x=0.5)

# Mover as configurações de fonte para os eixos
fig.update_xaxes(title_font=dict(size=16))
fig.update_yaxes(title_font=dict(size=16))

fig.update_yaxes(rangemode='tozero') # Começar o eixo Y em zero

fig.show()

#### Observação Variáveis Contratual/Financeira (Correlação com Churn):

*   **Tipo de Contrato**: Clientes com contrato **Mensal** apresentam uma correlação positiva forte com o Churn, indicando alta propensão a cancelar. Contratos **Anual** e **Bienal** têm correlações negativas, mostrando maior lealdade com contratos de longa duração.

*   **Fatura Digital**: Clientes que optam pela **fatura digital** têm correlação positiva com o Churn, sugerindo que podem ser mais propensos a cancelar do que aqueles com fatura impressa.

*   **Método de Pagamento**: O **Cheque eletrônico** tem uma correlação positiva significativa com o Churn, indicando que clientes que utilizam esse método são mais propensos a cancelar. Outros métodos de pagamento (Cartão de crédito, Transferência bancária) apresentam correlações negativas, associadas a menor Churn.

In [ ]:
# @title
import plotly.express as px

# 1. Calcular a matriz de correlação
# Certifique-se de que o nome da coluna de Churn está correto (ex: 'Churn' ou 'Churn_Num')
cols_num = ['Churn', 'tempo_contrato', 'valor_mensal', 'total_cobrado', 'contas_diarias']
matriz_corr = df_encoded[cols_num].corr()

custom_rb_weaker_red = [
    [0.0, 'rgb(99, 102, 255)'],    # Azul escuro (como o RdBu_r para valores negativos)
    [0.5, 'rgb(232, 232, 232)'], # Branco (para valores próximos de zero)
    [1.0, 'rgb(239, 85, 59)']  # Vermelho mais fraco (para valores positivos)
]

# 2. Criar o Heatmap
fig_heat = px.imshow(
    matriz_corr,
    text_auto='.2f',
    aspect="auto",
    color_continuous_scale=custom_rb_weaker_red,
    zmin=-1, zmax=1,
    title="Matriz de Correlação: Variáveis Numéricas vs Churn",
    labels=dict(color="Churn")
)

fig_heat.update_layout(width=1000, height=600,title_font_size=20,
    title_x=0.5)
fig_heat.show()

#### Observação Matriz de Correlação (Variáveis Numéricas vs Churn):

*   **Tempo de Contrato (Tenure)**: Apresenta uma correlação negativa moderada com o Churn, indicando que quanto mais tempo o cliente permanece, menor a probabilidade de churn. Além disso, tem uma forte correlação positiva com 'total_cobrado', o que é esperado, já que clientes com mais tempo de casa acumulam mais cobranças totais.

*   **Valor Mensal (Monthly Charges)**: Possui uma correlação positiva baixa com o Churn, sugerindo que valores mensais mais altos estão ligeiramente associados a uma maior propensão ao cancelamento. Também tem uma correlação positiva com 'total_cobrado' e 'contas_diarias'.

*   **Total Cobrado (Total Charges)**: Mostra uma correlação negativa baixa com o Churn, indicando que clientes que pagaram mais no total são menos propensos a cancelar. Isso é consistente com a correlação de 'tempo_contrato'.

*   **Contas Diárias (Daily Charges)**: Tem uma correlação positiva baixa com o Churn, similar ao 'valor_mensal', sugerindo que um custo diário percebido mais alto pode contribuir para a evasão.

In [ ]:
# @title
# Filtrar as correlações para as Serviços Infraestrutura
service_correlations = pd.Series({
    'tempo_contrato': correlation_with_all.loc['tempo_contrato'],
    'valor_mensal': correlation_with_all.loc['valor_mensal'],
    'total_cobrado': correlation_with_all.loc['total_cobrado'],
    'contas_diarias': correlation_with_all.loc['contas_diarias']
})

# Renomear as chaves para melhor visualização no gráfico
service_correlations = service_correlations.rename(index={
    'tempo_contrato': 'Tempo de Contrato (Tenure)',
    'valor_mensal': 'Valor Mensal',
    'total_cobrado': 'Cobrança Total',
    'contas_diarias': 'Contas Diárias'

})

custom_rb_weaker_red = [
    [0.0, 'rgb(99, 102, 255)'],    # Azul escuro (como o RdBu_r para valores negativos)
    [0.5, 'rgb(210, 210, 210)'], # Branco (para valores próximos de zero)
    [1.0, 'rgb(239, 85, 59)']  # Vermelho mais fraco (para valores positivos)
]


# Criar um DataFrame para o Plotly Express
df_service_corr = service_correlations.reset_index()
df_service_corr.columns = ['Análise Tenure/custos', 'Churn']

# Criar o gráfico de barras
fig = px.bar(df_service_corr, x='Análise Tenure/custos', y='Churn',
             title='Correlação Análise Tenure/Custos com Churn',
             color='Churn', # Cores baseadas no valor da correlação
             color_continuous_scale=custom_rb_weaker_red,
             range_color=[-0.3, 0.3], # Ajustar a escala de cores para melhor contraste
             text_auto=".2f") # Adiciona o valor da correlação nas barras

fig.update_layout(hoverlabel=dict(font_color="white"),
                  width=1000,
                  height=600,
                  title_font_size=20,
                  title_x=0.5)

# Mover as configurações de fonte para os eixos
fig.update_xaxes(title_font=dict(size=16))
fig.update_yaxes(title_font=dict(size=16))

fig.update_yaxes(rangemode='tozero') # Começar o eixo Y em zero

fig.show()

#### Observação Análise Tenure/Custos (Correlação com Churn):

*   **Tempo de Contrato (Tenure)**: Possui uma correlação negativa significativa com o Churn, indicando que quanto mais tempo o cliente permanece, menor a probabilidade de cancelar o serviço.

*   **Valor Mensal (Monthly Charges)**: Apresenta uma correlação positiva moderada com o Churn, sugerindo que valores mensais mais altos estão associados a uma maior propensão à evasão.

*   **Total Cobrado (Total Charges)**: Mostra uma correlação negativa com o Churn, o que significa que clientes que acumularam um valor total maior de cobranças são menos propensos a cancelar, reforçando a lealdade de longo prazo.

*   **Contas Diárias (Daily Charges)**: Tem uma correlação positiva com o Churn, similar ao valor mensal, indicando que um custo diário percebido mais alto pode ser um fator para o cancelamento.

In [ ]:
# @title
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig_scatter = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Mensal ($)", "Total ($)", "Diário ($)"),
    horizontal_spacing=0.06
)

vars_financeiras = ['valor_mensal', 'total_cobrado', 'contas_diarias']

for i, col in enumerate(vars_financeiras):
    for status, label, cor in zip([0, 1], ['Não', 'Sim'], ['#636EFA', '#EF553B']):
        mask = df_encoded['Churn'] == status

        # USANDO SCATTERGL PARA PERFORMANCE
        fig_scatter.add_trace(
            go.Scattergl(
                x=df_encoded[mask]['tempo_contrato'],
                y=df_encoded[mask][col],
                mode='markers',
                name=label if i == 0 else "",
                marker=dict(color=cor, opacity=0.3, size=4),
                showlegend=(i == 0)
            ), row=1, col=i+1
        )

fig_scatter.update_layout(height=500, width=1000, title_text="Dispersão: Tempo de Contrato (Tenure) vs Métricas de Custo",   title_font_size=20, title_x=0.5, template="plotly", legend_title_text="Churn")
fig_scatter.show()

#### Observação Gráfico de Dispersão (Tenure vs Custos):

*   **Churn em Clientes de Curto Prazo e Alto Custo Mensal**: Observa-se uma alta concentração de churn ('Sim', em vermelho/laranja) entre clientes com baixo 'tempo_contrato' (poucos meses) e 'valor_mensal' elevado. Isso reforça a ideia de que novos clientes que já enfrentam custos mensais altos são mais propensos a cancelar.

*   **Churn e Total Cobrado**: O churn é mais evidente em clientes com baixo 'tempo_contrato' e 'total_cobrado' baixo. À medida que o 'total_cobrado' aumenta, a presença de churn diminui, indicando que clientes com um histórico financeiro mais longo e consolidado são mais leais.

*   **Tenure e Estabilidade**: Clientes com 'tempo_contrato' mais longo tendem a ter menos churn, independentemente do 'valor_mensal' ou 'total_cobrado', sugerindo que a permanência prolongada está associada à satisfação e estabilidade do cliente.

*   **Relação Custo-Churn**: Os gráficos de 'valor_mensal' e 'contas_diarias' mostram que, em geral, quanto maior o custo do serviço (seja mensal ou diário), maior a dispersão de clientes com churn, especialmente nos estágios iniciais do contrato.

In [ ]:
from IPython.display import HTML

HTML("""
<style>
    /* VARIÁVEIS DE CORES DINÂMICAS PARA O JUPYTER LAB */
    :root {
        --cor-titulo: #3498db;
        --cor-texto: var(--jp-content-font-color1); /* Cor principal do tema */
        --cor-bg-box: #f1f7fc;
    }

    [data-jp-theme-light="false"] :root {
        --cor-titulo: #5dade2;
        --cor-bg-box: #1c2a35;
    }

    /* ESTILO DO TEXTO E TÍTULOS */
    .jp-RenderedMarkdown p, .jp-RenderedMarkdown li {
        font-size: 18px !important;
        color: var(--cor-texto) !important;
    }
    .jp-RenderedMarkdown h1 {
        font-size: 34px !important;
        color: var(--cor-titulo) !important;
        border-bottom: 2px solid var(--cor-titulo);
    }
    .jp-RenderedMarkdown h2 { color: var(--cor-titulo) !important; }

    /* CAIXA DE ANÁLISE (SUBSTITUTO DO >) */
    .analise-box {
        font-style: italic !important;
        border-left: 6px solid var(--cor-titulo) !important;
        background-color: var(--cor-bg-box) !important;
        color: var(--cor-texto) !important;
        padding: 15px !important;
        margin: 20px 0 !important;
        font-size: 18px !important;
        display: block !important;
    }
    .analise-box strong { color: var(--cor-titulo) !important; }

    /* ZOOM DIRECIONAL */
    .jp-RenderedMarkdown img.zoom-right, .jp-RenderedMarkdown img.zoom-left {
        transition: all 0.4s ease-in-out !important;
        position: relative !important;
        z-index: 1 !important;
        cursor: zoom-in !important;
    }
    .jp-RenderedMarkdown img.zoom-right:hover { transform: scale(1.8) !important; transform-origin: left center !important; z-index: 9999 !important; box-shadow: 0 10px 30px rgba(0,0,0,0.5) !important; }
    .jp-RenderedMarkdown img.zoom-left:hover { transform: scale(1.8) !important; transform-origin: right center !important; z-index: 9999 !important; box-shadow: 0 10px 30px rgba(0,0,0,0.5) !important; }

    /* AJUSTES DE CONTAINER */
    .jp-RenderedMarkdown, .jp-Cell-outputWrapper { overflow: visible !important; }
    .jp-RenderedMarkdown table { display: table !important; width: 100% !important; overflow: visible !important; }
</style>
""")

# 📈 Relatório Final: Análise de Churn Telecom X

Este relatório apresenta as principais descobertas da análise de `Churn` (evasão de clientes) da Telecom X. O objetivo é identificar os fatores que contribuem para o cancelamento de serviços e fornecer insights acionáveis para estratégias de retenção e aumento do LTV (Lifetime Value).

---

## 1. Visão Geral da Distribuição de Churn

### 1.1 Panorama da Base
Este gráfico estabelece a linha de base do projeto, mostrando a divisão entre clientes retidos e cancelados.

<br/>
<img src="images/Distribuicao_Churn.png" width="100%">

> **Análise:**
> * **Retenção (Não Churn)**: A maioria dos clientes (**73,4%**) permanece na base.
> * **Evasão (Churn)**: A taxa de **26,6%** é expressiva, indicando que aproximadamente um em cada quatro clientes cancela o serviço, demandando estratégias urgentes de retenção.

---

## 2. Fatores-Chave Influenciadores do Churn (Variáveis Categóricas)

### 2.1 Análise Demográfica

| Perfil Demográfico | Correlação Perfil Demográfico |
| :---: | :---: |
| <img src="images/Analise_Demografica.png" class="zoom-right" width="100%"/> | <img src="images/Corr_Variavel_Demográfica.png" class="zoom-left" width="100%"/> |

> **Análise:**
> * **Gênero:** Impacto **nulo**. A evasão é idêntica entre homens e mulheres, o que permite campanhas de retenção unificadas.
> * **Idosos (60+):** Grupo de **alto risco**. Apresentam maior propensão ao `Churn` (correlação positiva moderada), sendo um público que exige atenção especial.
> * **Vínculos Familiares:** Ter parceiros ou dependentes funciona como uma **barreira de saída**. Clientes solteiros e sem dependentes possuem menor lealdade e evadem com mais facilidade.

### 2.2 Análise Serviços Infraestrutura

| Serviços Infraestrutura | Correlação Serviços Infraestrutura |
| :---: | :---: |
| <img src="images/Servicos_Infra.png" class="zoom-right" width="100%"/> | <img src="images/Corr_Servicos_Infra.png" class="zoom-left" width="100%"/> |

> **Análise:**
> * **Telefonia:** Impacto **mínimo**. Ter uma ou múltiplas linhas não influencia significativamente a decisão de cancelamento.
> * **Internet de Fibra Óptica:** Ponto crítico de **atenção**. Clientes com Fibra apresentam a maior propensão ao `Churn`. Isso pode indicar insatisfação com preços ou que este público é mais visado por ofertas agressivas da concorrência.
> * **DSL e Sem Internet:** Grupos de **maior estabilidade**. Clientes que utilizam DSL ou que não possuem serviço de internet tendem a ser mais leais.

### 2.3 Análise Serviços Agregados

<br/>
<img src="images/Servicos_Agregados.png" width="100%">

> **Análise:**
> * **Serviços de Utilidade (Segurança, Backup e Suporte):** O impacto na retenção é **altíssimo**. Clientes que **NÃO** assinam esses serviços apresentam taxas de `Churn` superiores; esses produtos funcionam como "âncoras" de fidelidade.
> * **Serviços de Entretenimento (Streaming):** Também auxiliam na retenção, mas seu efeito é inferior aos serviços de segurança e suporte técnico.

### 2.4 Correlação Serviços Agregados

| Serviços Agregados | Ausência Serviços Agregados |
| :---: | :---: |
| <img src="images/Corr_Servicos_Agregados.png" class="zoom-right" width="100%"/> | <img src="images/Corr_Ausencia_Servicos.png" class="zoom-left" width="100%"/> |

> **Análise:**
> * **Barreiras de Retenção:** Existe uma forte **correlação negativa** entre a presença de serviços como Segurança Online e Suporte Técnico com o `Churn`.
> * **O Paradoxo do Streaming:** O uso de Streaming mostra uma correlação que sugere um perfil de usuário mais exigente: se o serviço não superar as expectativas ou o custo aumentar, eles tendem a trocar de provedor com facilidade.

### 2.5 Análise Contratual e Financeira

| Análise Contratual / Financeira | Correlação Contratual / Financeira |
| :---: | :---: |
| <img src="images/Analise_Contratual_Fianceira.png" class="zoom-right" width="100%"/> | <img src="images/Corr_Contratual_Fianceira.png"   class="zoom-left" width="100%"/> |

> **Análise Técnica:** > * **Tipo de Contrato:** O maior preditor de evasão. Contratos **Mensais** possuem correlação positiva drástica com o `Churn`, enquanto contratos **Bienais** garantem a maior lealdade.
> * **Método de Pagamento:** O **Cheque Eletrônico** é um ponto crítico de retenção. Métodos **Automáticos** (Cartão e Transferência) favorecem a permanência por reduzirem o atrito financeiro.
> * **Fatura Digital:** Clientes com fatura digital tendem a sair mais. Isso sugere que este perfil de cliente é mais digitalizado e ágil para realizar cancelamentos ou comparar ofertas online.

---

## 3. Fatores-Chave Influenciadores do Churn (Variáveis Numéricas)

### 3.1 Análise Tenure e Custos

| Tenure - Valor Mensal | Cobrança Total - Contas Diárias |
| :---: | :---: |
| <img src="images/Analise_Tenure_Custos_1.png" class="zoom-right" width="100%"/> | <img src="images/Analise_Tenure_Custos_2.png" class="zoom-left" width="100%"/> |

> **Análise:**
> * **Curva de Lealdade (Tenure):** O risco de evasão é altíssimo nos **primeiros meses**. Após o primeiro ano, a probabilidade de `Churn` cai drasticamente.
> * **Sensibilidade ao Preço:** Clientes com **Mensalidades** mais elevadas são mais propensos a cancelar, indicando sensibilidade ao custo mensal percebido.

### 3.2 Correlação Tenure e Custos

| Matriz Correlação (Heatmap) | Correlação Tenure / Custos |
| :---: | :---: |
| <img src="images/Matriz_corr.png" class="zoom-right" width="100%"/> | <img src="images/Corr_Tenure_Custo.png" class="zoom-left" width="100%"/> |

> **Análise:**
> * **Tempo de Casa (Tenure):** É o fator de proteção mais forte, com **correlação negativa significativa** com o `Churn`.
> * **Valor Acumulado (Total Charges):** Apresenta **correlação negativa** com o `Churn`; clientes que já construíram um histórico financeiro longo tendem a ser mais leais.

---

## 4. Conclusões e Insights Estratégicos

* **Foco na Fibra Ótica:** Priorizar a migração de clientes de Fibra Ótica para planos anuais com descontos progressivos para aumentar o custo de saída.
* **Pacotes "Âncora":** Incentivar a inclusão de **Suporte Técnico** e **Segurança Online**, pois estes serviços reduzem comprovadamente a evasão.
* **Régua de Retenção Precoce:** Implementar ações de satisfação e "onboarding" nos primeiros 6 meses de contrato.
* **Meios de Pagamento:** Incentivar a migração do "Cheque Eletrônico" para **Débito Automático** para reduzir pontos de atrito mensais.

---
---

Estas descobertas podem servir como ponto de partida para a equipe de Data Science aprofundar a modelagem preditiva de churn e para a equipe de negócios desenvolver estratégias eficazes de retenção de clientes.